In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

import numpy as np
import cv2
import random
import os

import os
import cv2
import numpy as np
import pandas as pd
from PIL import Image

import pandas as pd
import numpy as np
import cv2
from skimage import exposure
from scipy.ndimage import gaussian_filter
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, layers
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import LearningRateScheduler
import math
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import pandas as pd
import cv2
from skimage import exposure
from scipy.ndimage import gaussian_filter
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

In [ ]:
import numpy as np
import random
import tensorflow as tf
import torch

# Set seeds to ensure reproducibility
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)  # For multi-GPU setups

In [ ]:
# Load the dataset
data = pd.read_csv("../../aws_s3/DDR+APTOS_TRAIN_TEST/train_test_DDR_APTOS.csv")

filtered_data = data[data["Data_source"] == "DDR"]
# Split the dataset into training and testing sets
train_data = filtered_data[filtered_data["Split"] == "Train"]
test_data = filtered_data[filtered_data["Split"] == "Test"]

> Preprocessing

In [ ]:
# Define output directories
luminosity_output_dir_train = "luminosity_images_train/"
cropped_output_dir_train = "cropped_images_train/"
normalized_output_dir_train = "normalized_images_train/"
augmentation_output_dir_train = "augmented_images_train/"
luminosity_output_dir_test = "luminosity_images_test/"
cropped_output_dir_test = "cropped_images_test/"
normalized_output_dir_test = "normalized_images_test/"

# Create directories if they don't exist
os.makedirs(luminosity_output_dir_train, exist_ok=True)
os.makedirs(cropped_output_dir_train, exist_ok=True)
os.makedirs(normalized_output_dir_train, exist_ok=True)
os.makedirs(augmentation_output_dir_train, exist_ok=True)
os.makedirs(luminosity_output_dir_test, exist_ok=True)
os.makedirs(cropped_output_dir_test, exist_ok=True)
os.makedirs(normalized_output_dir_test, exist_ok=True)

In [ ]:
# luminosity and noise removal function
def luminosity_image(image_path, output_dir):
    # Load the image
    image = cv2.imread(image_path)

    # Convert to LAB color space
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)

    # Apply CLAHE to L channel
    clahe = cv2.createCLAHE(clipLimit=5.0, tileGridSize=(8, 8))
    l_eq = clahe.apply(l)

    # Merge back LAB channels
    lab_eq = cv2.merge((l_eq, a, b))
    enhanced_image = cv2.cvtColor(lab_eq, cv2.COLOR_LAB2BGR)

    # Image noise removal using Gaussian filter
    filtered_image = cv2.GaussianBlur(enhanced_image, (5, 5), 0)

    # Save the preprocessed image
    image_name = os.path.basename(image_path)
    output_path = os.path.join(output_dir, image_name)
    cv2.imwrite(output_path, filtered_image)

    return output_path

In [ ]:
def crop_image(img, threshold=15, resize_flag=False, desired_size=(224, 224)):
    img_np = np.array(img)
    x_dim = img_np.shape[0]
    y_dim = img_np.shape[1]
    pixel_sums = img_np.sum(axis=2)
    x_arr = pixel_sums.sum(axis=1)
    y_arr = pixel_sums.sum(axis=0)
    x_start = np.where(x_arr > threshold * y_dim)[0][0]
    x_end = np.where(x_arr > threshold * y_dim)[0][-1]
    y_start = np.where(y_arr > threshold * x_dim)[0][0]
    y_end = np.where(y_arr > threshold * x_dim)[0][-1]
    new_img = img_np[x_start:x_end, y_start:y_end]
    new_img = Image.fromarray(new_img)
    if resize_flag:
        new_img = new_img.resize(desired_size)
    return new_img

In [ ]:
def normalize_image(img):
    # Convert image to numpy array
    img_np = np.array(img)

    # Calculate mean and standard deviation (std) channel-wise
    mean_channels = np.mean(img_np, axis=(0, 1))
    std_channels = np.std(img_np, axis=(0, 1))

    # Normalize each channel separately
    normalized_image = np.zeros_like(img_np, dtype=np.float32)
    for channel in range(img_np.shape[2]):
        normalized_image[:, :, channel] = (
            img_np[:, :, channel] - mean_channels[channel]
        ) / std_channels[channel]

    # Scale values to be within [0, 255]
    normalized_image = (
        (normalized_image - np.min(normalized_image))
        / (np.max(normalized_image) - np.min(normalized_image))
        * 255
    )

    # Clip and return the normalized image
    normalized_image = np.clip(normalized_image, 0, 255)
    return normalized_image.astype(np.uint8)

In [ ]:
def rotate_image(image, angle):
    # Rotate the image by the specified angle
    height, width = image.shape[:2]
    rotation_matrix = cv2.getRotationMatrix2D((width / 2, height / 2), angle, 1)
    rotated_image = cv2.warpAffine(image, rotation_matrix, (width, height))
    return rotated_image


def flip_image(image):
    if random.choice([True, False]):
        image = cv2.flip(image, 1)
    if random.choice([True, False]):
        image = cv2.flip(image, 0)
    return image


def shear_image(image, shear_range):
    height, width = image.shape[:2]
    shear_value = random.uniform(-shear_range, shear_range)

    if shear_value < 0:
        shear_matrix = np.array([[1, -shear_value, 0], [0, 1, 0]])
    else:
        shear_matrix = np.array([[1, shear_value, 0], [0, 1, 0]])

    sheared_image = cv2.warpAffine(image, shear_matrix, (width, height))
    return sheared_image


def translate_image(image, translate_range):
    height, width = image.shape[:2]

    max_shift_x = int(width * 0.1)
    max_shift_y = int(height * 0.1)

    translate_x = random.randint(-max_shift_x, max_shift_x)
    translate_y = random.randint(-max_shift_y, max_shift_y)

    translation_matrix = np.array(
        [[1, 0, translate_x], [0, 1, translate_y]], dtype=np.float32
    )  # Ensure matrix type is float32

    translated_image = cv2.warpAffine(image, translation_matrix, (width, height))

    return translated_image


def adjust_brightness(image, brightness_range):
    brightness_factor = 1.0 + random.uniform(-brightness_range, brightness_range)
    adjusted_image = np.clip(image * brightness_factor, 0.25, 255).astype(np.uint8)
    return adjusted_image


def augmented_fn(image):
    angle = random.uniform(-35, 35)
    image = rotate_image(image, angle)
    image = flip_image(image)
    image = shear_image(image, shear_range=0.15)
    image = translate_image(image, translate_range=0.1)
    image = adjust_brightness(image, brightness_range=0.25)

    return image


def apply_augmentation(image):
    augmented_images = []
    for _ in range(20):
        augmented_image = augmented_fn(image)
        augmented_images.append(augmented_image)
    return np.array(augmented_images)

In [ ]:
X_train = []
y_train = []


# Function to get the correct image path with extension
def get_image_path(base_dir, img_name):
    for ext in ["jpg", "png"]:
        temp_path = f"{base_dir}/{img_name}.{ext}"
        if os.path.exists(temp_path):
            return temp_path
    return None  # Return None if no file is found


# Load and preprocess each image
for index, row in train_data.iterrows():
    img_name, label = row["Image_ID"], row["Retinopathy_Grade"]
    image_path = get_image_path(
        "../../aws_s3/DDR+APTOS_TRAIN_TEST/train_new", img_name
    )  # Dynamically get the correct image path

    if image_path is None:
        print(f"Image file for {img_name} not found in JPG or PNG format.")
        continue  # Skip this iteration if the file doesn't exist

    # Change Luminosity and do noise removal for the image
    luminosity_img_path = luminosity_image(image_path, luminosity_output_dir_train)

    # Crop and resize the image
    luminosity_img = Image.open(luminosity_img_path)
    cropped_resized_img = crop_image(
        luminosity_img, resize_flag=True, desired_size=(224, 224)
    )

    # Save cropped image
    cropped_img_name = f"{img_name}_{index+1}.jpg"
    cropped_img_path = os.path.join(cropped_output_dir_train, cropped_img_name)
    cropped_resized_img.save(cropped_img_path)

    # Normalize the image
    normalized_img = normalize_image(cropped_resized_img)

    # Save normalized image
    normalized_img_name = f"norm_{img_name}_{index+1}.jpg"
    normalized_img_path = os.path.join(normalized_output_dir_train, normalized_img_name)
    cv2.imwrite(normalized_img_path, cv2.cvtColor(normalized_img, cv2.COLOR_RGB2BGR))

    # Augment image 20 times
    augmented_images = apply_augmentation(normalized_img)

    for aug_index, augmented_img in enumerate(augmented_images):
        # Use aug_index to create a unique filename for each augmented image
        augmented_img_path = os.path.join(
            augmentation_output_dir_train, f"{img_name}_{index+1}_{aug_index+1}.jpg"
        )
        cv2.imwrite(augmented_img_path, cv2.cvtColor(augmented_img, cv2.COLOR_RGB2BGR))
        # Append to X_train and y_train
        X_train.append(augmented_img)
        y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

In [ ]:
X_test = []
y_test = []

# Load and preprocess each image
for index, row in test_data.iterrows():
    img_name, label = row["Image_ID"], row["Retinopathy_Grade"]
    image_path = get_image_path(
        "../../aws_s3/DDR+APTOS_TRAIN_TEST/test_new", img_name
    )  # Dynamically get the correct image path

    if image_path is None:
        print(f"Image file for {img_name} not found in JPG or PNG format.")
        continue  # Skip this iteration if the file doesn't exist

    # Preprocess the image
    luminosity_img_path = luminosity_image(image_path, luminosity_output_dir_test)

    # Crop and resize the image
    luminosity_img = Image.open(luminosity_img_path)
    cropped_resized_img = crop_image(
        luminosity_img, resize_flag=True, desired_size=(224, 224)
    )

    # Save cropped image
    cropped_img_name = f"{img_name}_{index+1}.jpg"
    cropped_img_path = os.path.join(cropped_output_dir_test, cropped_img_name)
    cropped_resized_img.save(cropped_img_path)

    # Normalize the image
    normalized_img = normalize_image(cropped_resized_img)

    # Save normalized image
    normalized_img_name = f"norm_{img_name}_{index+1}.jpg"
    normalized_img_path = os.path.join(normalized_output_dir_test, normalized_img_name)
    cv2.imwrite(normalized_img_path, cv2.cvtColor(normalized_img, cv2.COLOR_RGB2BGR))

    # Append to X_test and y_test
    X_test.append(normalized_img)
    y_test.append(label)

X_test = np.array(X_test)
y_test = np.array(y_test)

> Initialize base model with pre-trained weights 

In [ ]:
base_model = tf.keras.applications.EfficientNetB0(
    include_top=False,
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_tensor=None,
    input_shape=(224, 224, 3),
    pooling="avg",
)

> freeze the base model

In [ ]:
base_model.trainable = False

> Create a new model on top with dropout

In [ ]:
num_classes = 5
epochs = 10
# Define the inputs
inputs = tf.keras.Input(shape=(224, 224, 3))

# Ensure the base_model is running in inference mode.
x = base_model(inputs, training=False)

x = layers.Dropout(0.5)(x)  # Dropout layer with 50% dropout rate
# Adding FC (Fully Connected) layers
x = layers.Dense(1000)(x)
x = layers.Dropout(0.5)(x)  # Another Dropout layer with 50% dropout rate
# Adding FC (Fully Connected) layers
x = layers.Dense(500)(x)

# Adding a final layer with SoftMax activation for classification
outputs = layers.Dense(num_classes, activation="softmax")(x)

# Creating the model
model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.summary(show_trainable=True)

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━┓
┃ Layer (type)                ┃ Output Shape          ┃    Param # ┃ Trai… ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━┩
│ input_layer_1 (InputLayer)  │ (None, 224, 224, 3)   │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ efficientnetb0 (Functional) │ (None, 1280)          │  4,049,571 │   N   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dropout (Dropout)           │ (None, 1280)          │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dense (Dense)               │ (None, 1000)          │  1,281,000 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dropout_1 (Dropout)         │ (None, 1000)          │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dense_1 (Dense)             │ (None, 500)           │    500,500 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dense_2 (Dense)             │ (None, 5)             │      2,505 │   Y   │
└─────────────────────────────┴───────────────────────┴────────────┴───────┘

 Total params: 5,833,576 (22.25 MB)

 Trainable params: 1,784,005 (6.81 MB)

 Non-trainable params: 4,049,571 (15.45 MB)

> Train the model on new data

In [ ]:
from tensorflow.keras.optimizers import SGD

# base learning rate
base_learning_rate = 1e-4
# maximum learning rate
max_learning_rate = 1e-2

# Create an instance of SGD optimizer with initial learning rate
optimizer = SGD(learning_rate=base_learning_rate, momentum=0.9)

# create class weight
classes = np.unique(y_train)
class_weights = compute_class_weight("balanced", classes=classes, y=y_train)
class_weight_dict = dict(zip(classes, class_weights))


# create triangular schedule
def triangular_schedule(epoch):
    """Triangular learning rate scheduler."""
    cycle_length = 10  # Define the length of a cycle
    cycle = math.floor(1 + epoch / (2 * cycle_length))
    x = abs(epoch / cycle_length - 2 * cycle + 1)
    lr = base_learning_rate + (max_learning_rate - base_learning_rate) * max(0, (1 - x))
    return lr


# When fitting the model, include the learning rate scheduler callback
lr_scheduler = LearningRateScheduler(triangular_schedule)

In [ ]:
model.compile(
    optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

print("Fitting the top layer of the model")
model.fit(
    X_train,
    y_train,
    epochs=epochs,
    batch_size=32,
    class_weight=class_weight_dict,
    callbacks=[lr_scheduler],
)

Fitting the top layer of the model
Epoch 1/50
259/259 ━━━━━━━━━━━━━━━━━━━━ 144s 521ms/step - accuracy: 0.2552 - loss: 1.8620 - learning_rate: 1.0000e-04
Epoch 2/50
259/259 ━━━━━━━━━━━━━━━━━━━━ 128s 492ms/step - accuracy: 0.4041 - loss: 1.3910 - learning_rate: 0.0011
Epoch 3/50
259/259 ━━━━━━━━━━━━━━━━━━━━ 127s 488ms/step - accuracy: 0.4451 - loss: 1.2442 - learning_rate: 0.0021
Epoch 4/50
259/259 ━━━━━━━━━━━━━━━━━━━━ 143s 550ms/step - accuracy: 0.4715 - loss: 1.1526 - learning_rate: 0.0031
Epoch 5/50
259/259 ━━━━━━━━━━━━━━━━━━━━ 150s 578ms/step - accuracy: 0.4719 - loss: 1.1490 - learning_rate: 0.0041
Epoch 6/50
259/259 ━━━━━━━━━━━━━━━━━━━━ 174s 671ms/step - accuracy: 0.4921 - loss: 1.1665 - learning_rate: 0.0050
Epoch 7/50
259/259 ━━━━━━━━━━━━━━━━━━━━ 186s 717ms/step - accuracy: 0.4776 - loss: 1.2266 - learning_rate: 0.0060
Epoch 8/50
259/259 ━━━━━━━━━━━━━━━━━━━━ 166s 640ms/step - accuracy: 0.4762 - loss: 1.3456 - learning_rate: 0.0070
Epoch 9/50
259/259 ━━━━━━━━━━━━━━━━━━━━ 151s 584m

KeyboardInterrupt: 

> Fine tuning: unfreeze all or part of the base model and retrain the whole model end-to-end with a very low learning rate.

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Unfreeze the base model
base_model.trainable = True
model.summary(show_trainable=True)

# Re-instantiate the optimizer
optimizer = SGD(learning_rate=base_learning_rate, momentum=0.9)

model.compile(
    optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

print("Fitting the end-to-end model")
# Train end-to-end. Stop before overfit
model.fit(
    X_train,
    y_train,
    epochs=epochs,
    batch_size=32,
    class_weight=class_weight_dict,
    callbacks=[
        lr_scheduler,
    ],
)

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━┓
┃ Layer (type)                ┃ Output Shape          ┃    Param # ┃ Trai… ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━┩
│ input_layer_1 (InputLayer)  │ (None, 224, 224, 3)   │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ efficientnetb0 (Functional) │ (None, 1280)          │  4,049,571 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dropout (Dropout)           │ (None, 1280)          │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dense (Dense)               │ (None, 1000)          │  1,281,000 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dropout_1 (Dropout)         │ (None, 1000)          │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dense_1 (Dense)             │ (None, 500)           │    500,500 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dense_2 (Dense)             │ (None, 5)             │      2,505 │   Y   │
└─────────────────────────────┴───────────────────────┴────────────┴───────┘

 Total params: 7,617,583 (29.06 MB)

 Trainable params: 5,791,553 (22.09 MB)

 Non-trainable params: 42,023 (164.16 KB)

 Optimizer params: 1,784,007 (6.81 MB)

Fitting the end-to-end model
Epoch 1/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 52s 3s/step - accuracy: 0.2855 - loss: 1.6575 - learning_rate: 1.0000e-04
Epoch 2/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 42s 3s/step - accuracy: 0.2993 - loss: 1.5589 - learning_rate: 0.0011
Epoch 3/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 54s 4s/step - accuracy: 0.4041 - loss: 1.2903 - learning_rate: 0.0021
Epoch 4/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 46s 4s/step - accuracy: 0.4362 - loss: 1.0729 - learning_rate: 0.0031
Epoch 5/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 44s 3s/step - accuracy: 0.5126 - loss: 0.9374 - learning_rate: 0.0041
Epoch 6/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 44s 3s/step - accuracy: 0.6447 - loss: 0.7271 - learning_rate: 0.0050
Epoch 7/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 45s 3s/step - accuracy: 0.6550 - loss: 0.6703 - learning_rate: 0.0060
Epoch 8/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 44s 3s/step - accuracy: 0.7967 - loss: 0.4998 - learning_rate: 0.0070
Epoch 9/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 46s 4s/step - accuracy: 0.7761 - loss: 0.4945 - learning_rate: 0.

> Evaluate on test data

In [ ]:
print("Test dataset evaluation")
model.evaluate(X_test, y_test)

Test dataset evaluation
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 411ms/step - accuracy: 0.5608 - loss: 2.8218


[3.0512337684631348, 0.5242718458175659]